In [260]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [261]:
url = 'https://www.britannica.com/art/Grammy-Award'
response = requests.get(url)

In [262]:
# save the HTML to file
with open ('Grammy-Award.html', mode = 'wb') as file:
    file.write(response.content)

In [263]:
# work with html in memory
soup = BeautifulSoup(response.content, 'html.parser')

In [264]:
# variable definations
table_dict = {'year':[], 'record of the year':[], 'song of the year':[], 'album of the year':[], 'best new artist': []}
keys=['year', 'record of the year', 'song of the year', 'album of the year', 'best new artist']

for a in range(len(soup.table.find_all('tr'))):
    # narrow down on elements in each row of the table
    tr = soup.table.find_all('tr')[a]
    

    for i in range(len(tr.find_all('td'))):
        # narrow down on element in each column
        content = tr.find_all('td')[i].text
        
        #append into the dictionary
        table_dict[keys[i]].append(content)
    
   

In [265]:
table_dict['year'] = table_dict['year'][3:]

In [266]:
table_df = pd.DataFrame.from_dict(table_dict)

In [267]:
table_df.iloc[11,-1]

'\nCrosby, Stills and Nash\n'

#### cleaning

In [268]:
splitted_table_df = pd.DataFrame.from_dict(table_dict['year'])
key = keys[1:]
key_2 = ['artist_name record', 'artist_name song', 'artist_name album', 'artist_name best']
for i in range(len(key)):
    splitted_table_df[[key[i], key_2[i]]] = table_df[key[i]].str.rsplit(",",n= 1, expand = True)
    

In [269]:
splitted_table_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   0                   65 non-null     object
 1   record of the year  65 non-null     object
 2   artist_name record  65 non-null     object
 3   song of the year    65 non-null     object
 4   artist_name song    62 non-null     object
 5   album of the year   65 non-null     object
 6   artist_name album   65 non-null     object
 7   best new artist     65 non-null     object
 8   artist_name best    1 non-null      object
dtypes: object(9)
memory usage: 4.7+ KB


In [270]:
splitted_table_df[splitted_table_df['artist_name best'].notnull()]

,0,record of the year,artist_name record,song of the year,artist_name song,album of the year,artist_name album,best new artist,artist_name best
11,\n 1969\n,"\n ""Aquarius/Let the Sunshine In",""" the 5th Dimension\n","\n ""Games People Play",""" Joe South\n","\n Blood, Sweat and Tears, Blood",Sweat and Tears\n,\nCrosby,Stills and Nash\n


In [271]:
splitted_table_df.rename(columns = {0:'year'}, inplace = True)
splitted_table_df.drop('artist_name best', axis = 1, inplace = True)
splitted_table_df.fillna("", inplace = True)
splitted_table_df.iloc[11,-1] = 'Crosby, Stills and Nash'

In [272]:
column_name = splitted_table_df.columns.to_list()
for i in column_name:
    splitted_table_df[i] = splitted_table_df[i].apply(lambda x: re.sub(r'"', '', x).strip())

In [273]:
 splitted_table_df.columns = ['year',
 'record of the year',
 'artist_name record',
 'song of the year',
 'artist_name song',
 'album of the year',
 'artist_name album',
 'artist_name best']


#### Tidy data

In [288]:
# Prepare an empty title column for best_new_artist

splitted_table_df['best new artist'] = ''
variable_list = [ ['artist_name record', 'record of the year'],
                  ['artist_name song', 'song of the year'],
                  ['artist_name album', 'album of the year'],
                  ['artist_name best', 'best new artist']
                ]
combined_long_table = pd.DataFrame()

for i in range(len(variable_list)):
    var = variable_list[i]
    long = splitted_table_df.melt(id_vars= ['year', var[0]],
                       value_vars = var[1], 
                       var_name = 'award', 
                       value_name ='title')
    long.columns = ['year', 'artist', 'award', 'title']
    combined_long_table = pd.concat([combined_long_table, long], ignore_index = True)
    